**This notebook focuses on the effectiveness of PixelDefend against adversarial attacks on the MNIST and MARVEL datasets.**

**Load prerequisites**

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras_radam import RAdam

import cv2
import os

import numpy as np
import matplotlib.pyplot as plt


from art import config
from art.attacks.evasion import FastGradientMethod, DeepFool, ProjectedGradientDescent, SaliencyMapMethod, CarliniL2Method, NewtonFool, BasicIterativeMethod
from art.defences.preprocessor import PixelDefend
from art.defences.trainer import AdversarialTrainer
from art.estimators.classification import KerasClassifier
from art.utils import load_mnist

**Load MNIST dataset**

In [6]:
(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()
# x_train, y_train = x_train[:10000], y_train[:10000]
x_test, y_test = x_test[:500], y_test[:500]

**Modification: Disabling eager execution to enable Section 1, Step 3**

In [8]:
tf.compat.v1.disable_eager_execution()

**Section 1 - Attacks**

Step 1: Load/Create a classifier model 

In [11]:
model = load_model("/home/cyber/mnist_trained_model.h5")
# model = load_model("/home/cyber/marvel_trained_classifier.h5")

classifier = KerasClassifier(model=model, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)

Optional step: Train and save a model for future use

In [ ]:
# model.fit(x_train, y_train, batch_size=64, epochs=5, verbose=True)

In [10]:
# model.save("/home/cyber/mnist_trained_model.h5")

Step 2: Evaluate the ART classifier on benign test examples

In [12]:
predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)

print("Undefended classifier")
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

/home/cyber/miniconda3/envs/tf-gpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[128,32,26,26] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node activation_1_1/Relu}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[activation_5_1/Softmax/_611]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[128,32,26,26] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node activation_1_1/Relu}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 successful operations.
0 derived errors ignored.

Step 3: Craft adversarial examples

In [6]:
# Select crafting algorithm
# adv_crafter = SaliencyMapMethod(classifier=classifier, theta = 0.1, gamma=0.3, verbose=True)
# adv_crafter = BasicIterativeMethod(classifier, eps=0.1, eps_step=0.01, max_iter=30)
# adv_crafter = ProjectedGradientDescent(classifier, eps=0.1, eps_step=0.01, max_iter=30)
# adv_crafter = NewtonFool(classifier=classifier, eta=0.005, max_iter=25, verbose=True)
# adv_crafter = DeepFool(classifier=classifier, epsilon=0.3e-12, max_iter=20)
# adv_crafter = CarliniL2Method(classifier=classifier, max_iter=20, verbose=True)
adv_crafter = FastGradientMethod(estimator=classifier, eps=0.3)

#x_train_adv = adv_crafter.generate(x_train)
x_test_adv = adv_crafter.generate(x_test)


/home/cyber/miniconda3/envs/tf-gpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Step 4: Evaluate the ART classifier on the adversarial test set

In [ ]:
predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Undefended classifier")
print("Accuracy on adversarial test examples: {}%".format(accuracy * 100))

Optional step: Plot images and their adversarial counterparts for reference

In [ ]:
num = 10
images = x_test[:num]
adv_examples = x_test_adv[:num]
#labels = y_train[:num]

#plot images
plt.figure(figsize=(10, 10))

for i in range(num):
    ax = plt.subplot(4, 5, i + 1)
    plt.imshow(images[i], cmap='gray')
    plt.axis("off")
    ax = plt.subplot(4, 5, i + 11)
    plt.imshow(adv_examples[i], cmap='gray')
    plt.axis("off")
    
    #plt.title('Label: {:}'.format(labels[i]))
    #ax.set_title('Label: {}'.format(labels[i]))
    
plt.tight_layout()
plt.show()

**Section 2 - Metrics**

Metric 1: Empirical Robustness

In [ ]:
attack_name = "fgsm"
score = empirical_robustness(classifier=classifier, x=x_test, attack_name=attack_name)
print("Empirical robustness of the classifier against {}: {}".format(attack_name, score))

Metric 2: CLEVER

In [ ]:
score = clever_u(classifier=classifier, x=x_test[1], nb_batches=10, batch_size=64, radius=25,norm=2)
print("CLEVER score of the classifier against an untargeted attack: {}".format(score))

**Section 3 - Defence**

Save original adversarial examples for reference

In [9]:
x_test_adv_original = x_test_adv

Creating a seperate 'robust' classifier, to enable side-by-sde analysis of both the original and defended model

In [ ]:
model = load_model("/home/cyber/basicmodel.h5")
robust_classifier = KerasClassifier(model=model, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)

Defence 1: Preprocessing

PixelCNN

In [ ]:
pixel_cnn = load_model("/home/cyber/pixelcnn_mnist.h5")

In [3]:
# Build a small Pixel CNN++ model to train on MNIST.

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
# from tensorflow_probability.distributions import PixelCNN

tfd = tfp.distributions
tfk = tf.keras
tfkl = tf.keras.layers

# tf.compat.v1.enable_v2_behavior()

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# (x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()
x_train, y_train = x_train[:5000], y_train[:5000]
x_test, y_test = x_test[:500], y_test[:500]

# # Load MNIST from tensorflow_datasets
# data = tfds.load('mnist')
# train_data, test_data = data['train'], data['test']

# def image_preprocess(x):
#   x['image'] = tf.cast(x['image'], tf.float32)
#   return (x['image'],)  # (input, output) of the model

# batch_size = 16
# train_it = train_data.map(image_preprocess).batch(batch_size).shuffle(1000)

image_shape = (28, 28, 1)
# Define a Pixel CNN network
dist = tfd.PixelCNN(
    image_shape=image_shape,
    num_resnet=1,
    num_hierarchies=2,
    num_filters=32,
    num_logistic_mix=5,
    dropout_p=.3,
)

# Define the model input
image_input = tfkl.Input(shape=image_shape)

# Define the log likelihood for the loss fn
log_prob = dist.log_prob(image_input)

# Define the model
model = tfk.Model(inputs=image_input, outputs=log_prob)
model.add_loss(-tf.reduce_mean(log_prob))

# Compile and train the model
model.compile(
    optimizer=tfk.optimizers.Adam(.001),
    metrics=[])

model.fit(x_train, y_train, epochs=10, verbose=True)


Epoch 1/10
157/157 [==============================] - 46s 168ms/step - loss: 1026.3543
Epoch 2/10
157/157 [==============================] - 17s 111ms/step - loss: 724.3569
Epoch 3/10
157/157 [==============================] - 17s 111ms/step - loss: 690.8315
Epoch 4/10
157/157 [==============================] - 18s 112ms/step - loss: 672.7293
Epoch 5/10
157/157 [==============================] - 17s 111ms/step - loss: 666.8042
Epoch 6/10
157/157 [==============================] - 18s 111ms/step - loss: 655.8241
Epoch 7/10
157/157 [==============================] - 18s 113ms/step - loss: 651.6655
Epoch 8/10
157/157 [==============================] - 17s 111ms/step - loss: 645.9012
Epoch 9/10
157/157 [==============================] - 15s 95ms/step - loss: 639.4327
Epoch 10/10
157/157 [==============================] - 13s 80ms/step - loss: 632.0571


Step 1: Input transformation

In [1]:
# defence = TotalVarMin(verbose = True)
defence = SpatialSmoothing(window_size=5)
defence = PixelDefend(clip_values=(min_pixel_value, max_pixel_value), eps=16, pixel_cnn = pixel_cnn, verbose=True)
# defence = FeatureSqueezing(clip_values=(min_pixel_value, max_pixel_value), bit_depth=8)

x_test_adv = defence(x_test_adv_original * 255)[0] / 255

#what does * 255)[0] / 255 do?

Additional step: Plot images and their adversarial counterparts, pre- and post-transformed, for reference

In [ ]:
num = 10
images = x_test[:num]
transformed_adv_examples = x_test_adv[:num]
pretransformed_adv_examples = x_test_adv_original[:num]
#labels = y_train[:num]

#plot images
plt.figure(figsize=(10, 10))

for i in range(num):
    ax = plt.subplot(6, 5, i + 1)
    plt.imshow(images[i], cmap='gray')
    plt.axis("off")
    
    ax = plt.subplot(6, 5, i + 11)
    plt.imshow(pretransformed_adv_examples[i], cmap='gray')
    plt.axis("off")
    
    ax = plt.subplot(6, 5, i + 21)
    plt.imshow(transformed_adv_examples[i], cmap='gray')
    plt.axis("off")
    
    #plt.title('Label: {:}'.format(labels[i]))
    #ax.set_title('Label: {}'.format(labels[i]))
    
plt.tight_layout()
plt.show()

Defence 2: Adversarial Training

Adversarial Training: Method 1 - Data Generators

Step 1: Build a Keras image augmentation object and wrap it in ART

In [ ]:
batch_size = 50
datagen = ImageDataGenerator(
    width_shift_range=0.125, height_shift_range=0.125, fill_mode="constant", cval=0.0
)
datagen.fit(x_train)
art_datagen = KerasDataGenerator(
    datagen.flow(x=x_train, y=y_train, batch_size=batch_size, shuffle=True),
    size=x_train.shape[0],
    batch_size=batch_size,
)

Step 2: Create adversarial trainer

In [ ]:
#adv_crafter = BasicIterativeMethod(robust_classifier, eps=0.3, eps_step=0.01, max_iter=40)
adv_trainer = AdversarialTrainer(classifier=robust_classifier, attacks=adv_crafter, ratio=1.0)

Step 3: Perform adversarial training

In [ ]:
#adv_trainer.fit_generator(generator=art_datagen, nb_epochs=5)
adv_trainer.fit(x_train, y_train, nb_epochs=5, batch_size=50)

Optional step: Save the adversarially trained classifier for future use

In [ ]:
#classifier.save("/home/cyber/pdg_defended_model.h5")

Adversarial Training: Alternative method - Load an existing adversarially trained model

In [ ]:
# path = get_file('mnist_cnn_robust.h5', extract=False, path=config.ART_DATA_PATH,
#                 url='https://www.dropbox.com/s/yutsncaniiy5uy8/mnist_cnn_robust.h5?dl=1')
# robust_classifier_model = load_model(path)
# robust_classifier = KerasClassifier(clip_values=(min_pixel_value, max_pixel_value), model=robust_classifier_model, use_logits=False)

Defence: Evaluation of effectiveness

Step 1: Evaluate the defended model on the clean test set

In [ ]:
predictions = robust_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Defended classifier")
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Step 2: Evaluate the defended model on the original adversarial test set

In [ ]:
predictions = robust_classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Defended classifier")
print("Accuracy on original adversarial test examples: {}%".format(accuracy * 100))

Step 3: Evaluate the defended model on new adversarial examples produced on the defended model

In [ ]:
#adv_crafter = SaliencyMapMethod(classifier=robust_classifier, theta = 1, verbose=True)
#adv_crafter = ProjectedGradientDescent(robust_classifier, eps=0.2)
adv_crafter = FastGradientMethod(estimator=robust_classifier, eps=0.3)
#adv_crafter = DeepFool(classifier=robust_classifier, epsilon=0.2)
#adv_crafter = NewtonFool(classifier=robust_classifier, verbose=True)
#adv_crafter = CarliniL2Method(classifier=robust_classifier, verbose=True)
#adv_crafter = BasicIterativeMethod(robust_classifier, eps=0.3, eps_step=0.01, max_iter=40)

x_test_adv_new = adv_crafter.generate(x_test)

#The following line is for when a preprocessing defence is used
#x_test_adv = defence(x_test_adv * 255)[0] / 255

predictions = robust_classifier.predict(x_test_adv_new)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Defended classifier")
print("Accuracy on new adversarial test examples: {}%".format(accuracy * 100))

Optional step: Plot images, their original adversarial counterparts, and their new adversarial counterparts for reference

In [ ]:
num = 10
images = x_test[:num]
adv_examples_new = x_test_adv_new[:num]
#labels = y_train[:num]

#plot images
plt.figure(figsize=(10, 10))

for i in range(num):
    ax = plt.subplot(6, 5, i + 1)
    plt.imshow(images[i], cmap='gray')
    plt.axis("off")
    ax = plt.subplot(6, 5, i + 11)
    plt.imshow(adv_examples[i], cmap='gray')
    plt.axis("off")
    ax = plt.subplot(6, 5, i + 21)
    plt.imshow(adv_examples_new[i], cmap='gray')
    plt.axis("off")
    
    #plt.title('Label: {:}'.format(labels[i]))
    #ax.set_title('Label: {}'.format(labels[i]))
 
plt.tight_layout()
plt.show()

Step 4: Compare the performance of the original and the robust classifier against FGSM over a range of eps values

In [ ]:
eps_range = [0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
accuracy_original = []
accuracy_robust = []

adv_crafter = FastGradientMethod(classifier)
adv_crafter_robust = FastGradientMethod(robust_classifier)

for eps in eps_range:
    adv_crafter.set_params(**{'eps': eps})
    adv_crafter_robust.set_params(**{'eps': eps})
    x_test_adv = adv_crafter.generate(x_test[:100])
    x_test_adv_robust = adv_crafter_robust.generate(x_test[:100])
    
    predictions_original = np.argmax(classifier.predict(x_test_adv), axis=1)
    accuracy_original += [np.sum(predictions_original == np.argmax(y_test[:100], axis=1))]
    
    predictions_robust = np.argmax(robust_classifier.predict(x_test_adv_robust), axis=1)
    accuracy_robust += [np.sum(predictions_robust == np.argmax(y_test[:100], axis=1))]

eps_range = eps_range

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.array(eps_range), np.array(accuracy_original), 'b--', label='Original classifier')
ax.plot(np.array(eps_range), np.array(accuracy_robust), 'r--', label='Robust classifier')

legend = ax.legend(loc='upper right', shadow=True, fontsize='large')
#legend.get_frame().set_facecolor('#00FFCC')

plt.xlabel('Attack strength (eps)')
plt.ylabel('Accuracy (%)')
plt.show()